In [105]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


# **Load Dataset**

In [106]:
TARGET_COL = 'diabetes_mellitus'

#Load Data
df_train = pd.read_csv('/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv',index_col=0)
df_test = pd.read_csv('/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv',index_col=0)
df_dict = pd.read_csv('/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv')
df_sample = pd.read_csv('/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv')
df_template = pd.read_csv('/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv')

#Display
display(df_train.head())

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,...,337.0,337.0,0,0,0,0,0,0,0,0
5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0


# EDA

In [107]:
print(df_train.shape)
print(df_test.shape)

(130157, 180)
(10234, 179)


In [108]:
print(df_train.nunique())
print(df_train.dtypes)

encounter_id                   130157
hospital_id                       204
age                                75
bmi                             41453
elective_surgery                    2
                                ...  
immunosuppression                   2
leukemia                            2
lymphoma                            2
solid_tumor_with_metastasis         2
diabetes_mellitus                   2
Length: 180, dtype: int64
encounter_id                     int64
hospital_id                      int64
age                            float64
bmi                            float64
elective_surgery                 int64
                                ...   
immunosuppression                int64
leukemia                         int64
lymphoma                         int64
solid_tumor_with_metastasis      int64
diabetes_mellitus                int64
Length: 180, dtype: object


In [109]:
# Target Variable Distribution in Training
df_train.groupby('diabetes_mellitus').encounter_id.count()

diabetes_mellitus
0    102006
1     28151
Name: encounter_id, dtype: int64

In [110]:
#Drop the id columns
id_cols = ['encounter_id','hospital_id','icu_id']
test_id = df_test['encounter_id']
df_train.drop(id_cols, axis=1, inplace=True)
df_test.drop(id_cols, axis=1, inplace=True)

In [111]:
# Drop columns that have over 80% missing value
drop_cols = ['h1_diasbp_invasive_max', 'h1_diasbp_invasive_min',
       'h1_mbp_invasive_max', 'h1_mbp_invasive_min',
       'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'h1_albumin_max',
       'h1_albumin_min', 'h1_bilirubin_max', 'h1_bilirubin_min',
       'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 'h1_calcium_min',
       'h1_creatinine_max', 'h1_creatinine_min', 'h1_hco3_max',
       'h1_hco3_min', 'h1_lactate_max', 'h1_lactate_min',
       'h1_platelets_max', 'h1_platelets_min', 'h1_wbc_max', 'h1_wbc_min',
       'h1_arterial_pco2_max', 'h1_arterial_pco2_min',
       'h1_arterial_ph_max', 'h1_arterial_ph_min', 'h1_arterial_po2_max',
       'h1_arterial_po2_min', 'h1_pao2fio2ratio_max',
       'h1_pao2fio2ratio_min']
df_train.drop(drop_cols, axis=1, inplace=True)
df_test.drop(drop_cols, axis=1, inplace=True)

In [112]:
print(df_train['readmission_status'].unique())
print(df_test['readmission_status'].unique())

# Drop readmission_status column
df_train.drop("readmission_status", axis=1, inplace=True)
df_test.drop("readmission_status", axis=1, inplace=True)

[0]
[0]


In [113]:
# Drop Correlated Columns
cor_cols = ['paco2_for_ph_apache', 'h1_inr_max', 'h1_inr_min']
df_train.drop(cor_cols, axis=1, inplace=True)
df_test.drop(cor_cols, axis=1, inplace=True)

In [114]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
age,125169.0,61.995103,16.822880,0.000000,52.000000,64.000000,75.000000,89.000000
bmi,125667.0,29.110260,8.262776,14.844926,23.598006,27.564749,32.803127,67.814990
elective_surgery,130157.0,0.189840,0.392176,0.000000,0.000000,0.000000,0.000000,1.000000
height,128080.0,169.607219,10.833085,137.200000,162.500000,170.100000,177.800000,195.590000
pre_icu_los_days,130157.0,0.839933,2.485337,-0.250000,0.045833,0.155556,0.423611,175.627778
...,...,...,...,...,...,...,...,...
immunosuppression,130157.0,0.025669,0.158146,0.000000,0.000000,0.000000,0.000000,1.000000
leukemia,130157.0,0.007307,0.085166,0.000000,0.000000,0.000000,0.000000,1.000000
lymphoma,130157.0,0.004187,0.064574,0.000000,0.000000,0.000000,0.000000,1.000000
solid_tumor_with_metastasis,130157.0,0.020852,0.142888,0.000000,0.000000,0.000000,0.000000,1.000000


In [115]:
missing_count = df_train.isna().sum()
missing_df = (pd.concat([missing_count.rename('Missing count'),
                     missing_count.div(len(df_train))
                          .rename('Missing ratio')],axis = 1)
             .loc[missing_count.ne(0)])
missing_df

,Missing count,Missing ratio
age,4988,0.038323
bmi,4490,0.034497
ethnicity,1587,0.012193
gender,66,0.000507
height,2077,0.015958
...,...,...
d1_arterial_ph_min,84807,0.651575
d1_arterial_po2_max,84010,0.645451
d1_arterial_po2_min,84010,0.645451
d1_pao2fio2ratio_max,93339,0.717126


In [116]:
"""
plt.figure(figsize = (16,8))
plt.title('Hospital Admit Source', size = 20)
sns.countplot(y ='hospital_admit_source', data = df_train); 
"""

"\nplt.figure(figsize = (16,8))\nplt.title('Hospital Admit Source', size = 20)\nsns.countplot(y ='hospital_admit_source', data = df_train); \n"

In [117]:
"""
plt.figure(figsize = (16,8))
plt.title('ICU Admit Source', size = 20)
sns.countplot(y ='icu_admit_source', data = df_train)
"""

"\nplt.figure(figsize = (16,8))\nplt.title('ICU Admit Source', size = 20)\nsns.countplot(y ='icu_admit_source', data = df_train)\n"

# Encoding & Imputation


In [118]:
## Print the categorical columns
print([col for col in df_train.columns if (1<df_train[col].nunique()) & (df_train[col].dtype != np.number)& (df_train[col].dtype != int) ])

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']


In [119]:
categorical_cols =  ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']

## Handle na values
df_train[categorical_cols] = df_train[categorical_cols].fillna("NA")
df_test[categorical_cols] = df_test[categorical_cols].fillna("NA")

df_train[categorical_cols].isna().sum()

ethnicity                0
gender                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
dtype: int64

In [120]:
for i in range(len(df_dict)):
    if str(df_dict['Data Type'][i])=="string":
        print(df_dict['Variable Name'][i])

bmi
ethnicity
gender
hospital_admit_source
icu_admit_source
icu_admit_type
icu_stay_type
icu_type
apache_2_diagnosis
apache_3j_diagnosis


In [121]:
print(df_train['apache_2_diagnosis'].nunique())
print(df_train['apache_3j_diagnosis'].nunique())

44
400


In [122]:
diagnosis_cols = ['apache_2_diagnosis','apache_3j_diagnosis']

In [123]:
for i in range(len(df_dict)):
    if str(df_dict['Data Type'][i])=="binary":
        print(df_dict['Variable Name'][i])

elective_surgery
readmission_status
apache_post_operative
arf_apache
gcs_unable_apache
intubated_apache
ventilated_apache
aids
cirrhosis
hepatic_failure
immunosuppression
leukemia
lymphoma
solid_tumor_with_metastasis
diabetes_mellitus


In [124]:
binary_cols = ['elective_surgery',
'apache_post_operative',
'arf_apache',
'gcs_unable_apache',
'intubated_apache',
'ventilated_apache',
'aids',
'cirrhosis',
'hepatic_failure',
'immunosuppression',
'leukemia',
'lymphoma',
'solid_tumor_with_metastasis']

In [125]:
#impute weight.
df_train.groupby('gender')["weight"].mean()

gender
F     77.042256
M     89.487018
NA    83.726304
Name: weight, dtype: float64

In [126]:
df_test.groupby('gender')["weight"].mean()

gender
F     77.112713
M     88.836120
NA    98.640000
Name: weight, dtype: float64

In [127]:
"""
# impute weight wrt gender
df_train["weight"] = np.where(df_train['gender'] == 'F', df_train["weight"].fillna(77.04), df_train["weight"])
df_train["weight"] = np.where(df_train['gender'] == 'M', df_train["weight"].fillna(89.49), df_train["weight"])
df_train["weight"] = np.where(df_train['gender'] == 'NA', df_train["weight"].fillna(83.73), df_train["weight"])

df_test["weight"] = np.where(df_test['gender'] == 'F', df_test["weight"].fillna(77.11), df_test["weight"])
df_test["weight"] = np.where(df_test['gender'] == 'M', df_test["weight"].fillna(88.84),df_test["weight"])
df_test["weight"] = np.where(df_test['gender'] == 'NA', df_test["weight"].fillna(98.64),df_test["weight"])
"""

'\n# impute weight wrt gender\ndf_train["weight"] = np.where(df_train[\'gender\'] == \'F\', df_train["weight"].fillna(77.04), df_train["weight"])\ndf_train["weight"] = np.where(df_train[\'gender\'] == \'M\', df_train["weight"].fillna(89.49), df_train["weight"])\ndf_train["weight"] = np.where(df_train[\'gender\'] == \'NA\', df_train["weight"].fillna(83.73), df_train["weight"])\n\ndf_test["weight"] = np.where(df_test[\'gender\'] == \'F\', df_test["weight"].fillna(77.11), df_test["weight"])\ndf_test["weight"] = np.where(df_test[\'gender\'] == \'M\', df_test["weight"].fillna(88.84),df_test["weight"])\ndf_test["weight"] = np.where(df_test[\'gender\'] == \'NA\', df_test["weight"].fillna(98.64),df_test["weight"])\n'

In [128]:
#impute height.
df_train.groupby('gender')["height"].mean()

gender
F     161.631668
M     176.340292
NA    171.425581
Name: height, dtype: float64

In [129]:
df_test.groupby('gender')["height"].mean()

gender
F     161.534660
M     175.836729
NA    175.160000
Name: height, dtype: float64

In [130]:
"""
# impute height wrt gender
df_train["height"] = np.where(df_train['gender'] == 'F', df_train["height"].fillna(161.631668), df_train["height"])
df_train["height"] = np.where(df_train['gender'] == 'M', df_train["height"].fillna(176.340292), df_train["height"])
df_train["height"] = np.where(df_train['gender'] == 'NA', df_train["height"].fillna(171.425581), df_train["height"])

df_test["height"] = np.where(df_test['gender'] == 'F', df_test["height"].fillna(161.534660), df_test["height"])
df_test["height"] = np.where(df_test['gender'] == 'M', df_test["height"].fillna(175.836729),df_test["height"])
df_test["height"] = np.where(df_test['gender'] == 'NA', df_test["height"].fillna(175.160000),df_test["height"])
"""

'\n# impute height wrt gender\ndf_train["height"] = np.where(df_train[\'gender\'] == \'F\', df_train["height"].fillna(161.631668), df_train["height"])\ndf_train["height"] = np.where(df_train[\'gender\'] == \'M\', df_train["height"].fillna(176.340292), df_train["height"])\ndf_train["height"] = np.where(df_train[\'gender\'] == \'NA\', df_train["height"].fillna(171.425581), df_train["height"])\n\ndf_test["height"] = np.where(df_test[\'gender\'] == \'F\', df_test["height"].fillna(161.534660), df_test["height"])\ndf_test["height"] = np.where(df_test[\'gender\'] == \'M\', df_test["height"].fillna(175.836729),df_test["height"])\ndf_test["height"] = np.where(df_test[\'gender\'] == \'NA\', df_test["height"].fillna(175.160000),df_test["height"])\n'

In [131]:
# we can fill the missing values of d1_pao2fio2ratio_max with pao2_apache/fio2_apache

df_train["d1_pao2fio2ratio_max"] = np.where((df_train["pao2_apache"].notna() 
                                             & df_train["fio2_apache"].notna()
                                             & df_train["d1_pao2fio2ratio_max"].isna() ), 
                                            df_train["pao2_apache"] / df_train["fio2_apache"], 
                                            df_train["d1_pao2fio2ratio_max"])
df_test["d1_pao2fio2ratio_max"] = np.where((df_test["pao2_apache"].notna() 
                                             & df_test["fio2_apache"].notna()
                                             & df_test["d1_pao2fio2ratio_max"].isna() ), 
                                            df_test["pao2_apache"] / df_test["fio2_apache"], 
                                            df_test["d1_pao2fio2ratio_max"])

In [132]:
all_features = df_test[[col for col in df_test if TARGET_COL != col]]
cat_cols = categorical_cols + binary_cols + diagnosis_cols
numerical_cols = [col for col in all_features if col not in cat_cols]

In [133]:
from sklearn.impute import SimpleImputer
# impute remaining categorical (binary and diagnosis) columns by mode

def cat_imputation(df):    
    imputer_cat = SimpleImputer(strategy='most_frequent')
    df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])
    return df

df_train = cat_imputation(df_train)
df_test = cat_imputation(df_test)


In [134]:
# impute remaining numerical columns by mean
for n in numerical_cols:
    df_train[n] = df_train.groupby(['ethnicity','gender'])[n].apply(lambda x: x.fillna(x.mean()))
    df_test[n] = df_test.groupby(['ethnicity','gender'])[n].apply(lambda x: x.fillna(x.mean()))

In [135]:
df_train.groupby('gender')["height"].mean()

gender
F     161.632193
M     176.339539
NA    171.064848
Name: height, dtype: float64

In [136]:
from sklearn.preprocessing import OrdinalEncoder

for c in categorical_cols:
    df_train[c] = OrdinalEncoder(dtype="int").fit_transform(df_train[[c]])
    df_test[c] = OrdinalEncoder(dtype="int").fit_transform(df_test[[c]])

In [137]:
"""
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler

fill_knn_train = KNN(k=3).fit_transform(df_train)
fill_knn_test = KNN(k=3).fit_transform(df_test)
df_train = pd.DataFrame(fill_knn_train)
df_test = pd.DataFrame(fill_knn_test)
"""

'\nfrom fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler\n\nfill_knn_train = KNN(k=3).fit_transform(df_train)\nfill_knn_test = KNN(k=3).fit_transform(df_test)\ndf_train = pd.DataFrame(fill_knn_train)\ndf_test = pd.DataFrame(fill_knn_test)\n'

In [138]:
"""
!pip install fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
def imputation(df):
    all_features = df[[col for col in df if TARGET_COL != col]]
    numerical_cols = [col for col in all_features if col not in categorical_cols]
    df = KNN(k=3).fit_transform(df)
    return df

df_train = imputation(df_train)
df_test = imputation(df_test)
"""

'\n!pip install fancyimpute\nfrom fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler\ndef imputation(df):\n    all_features = df[[col for col in df if TARGET_COL != col]]\n    numerical_cols = [col for col in all_features if col not in categorical_cols]\n    df = KNN(k=3).fit_transform(df)\n    return df\n\ndf_train = imputation(df_train)\ndf_test = imputation(df_test)\n'

In [139]:

compression_opts = dict(method='zip',
                        archive_name='out.csv')  
df_train.to_csv('tarin_out.zip', index=False,
          compression=compression_opts)  
df_test.to_csv('test_out.zip', index=False,
          compression=compression_opts)  


# Deal with Imblanaced Data

In [140]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve,auc, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.ensemble import BalancedBaggingClassifier, BalancedRandomForestClassifier, RUSBoostClassifier

In [141]:
X_train, X_valid, y_train, y_valid = train_test_split(
     df_train[[c for c in df_train if TARGET_COL != c]], df_train[TARGET_COL], test_size=0.20, random_state=999)
print(X_train.shape,X_valid.shape)

(104125, 140) (26032, 140)


In [142]:
rdm = RandomOverSampler(random_state=999)
X_rdm, y_rdm = rdm.fit_resample(X_train, y_train)

In [143]:
"""
model = XGBClassifier(tree_method = 'gpu_hist',learning_rate=0.45,n_estimators=164, max_depth=8, min_child_weight=2, random_state=999) 
model.fit(X_rdm, y_rdm)
features = X_train.columns
print(f"AUC is {roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])}")
print(metrics.classification_report(y_valid, model.predict(X_valid), labels=[0, 1]))
"""

'\nmodel = XGBClassifier(tree_method = \'gpu_hist\',learning_rate=0.45,n_estimators=164, max_depth=8, min_child_weight=2, random_state=999) \nmodel.fit(X_rdm, y_rdm)\nfeatures = X_train.columns\nprint(f"AUC is {roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])}")\nprint(metrics.classification_report(y_valid, model.predict(X_valid), labels=[0, 1]))\n'

In [159]:
model = LGBMClassifier(random_state=999, tree_method = 'gpu_hist',boosting_type='gbdt', 
                       num_leaves=64, learning_rate = 0.1,n_estimators = 125, max_depth =10,
                       min_child_samples=20, min_child_weight=0.002,
                       reg_alpha=0.0001, reg_lambda=0.0001)
model.fit(X_rdm, y_rdm)

features = X_train.columns
print(f"AUC is {roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])}")
print(metrics.classification_report(y_valid, model.predict(X_valid), labels=[0, 1]))

AUC is 0.8516107191246647
              precision    recall  f1-score   support

           0       0.92      0.80      0.86     20463
           1       0.50      0.76      0.60      5569

    accuracy                           0.79     26032
   macro avg       0.71      0.78      0.73     26032
weighted avg       0.83      0.79      0.80     26032



In [160]:
X_test = df_test
y_test = model.predict_proba(X_test)[:,1]
print(y_test)
submission = pd.DataFrame({
        "encounter_id": test_id,
        "diabetes_mellitus": y_test
        })
submission = submission.sort_values(by=['encounter_id'])
print(submission)
submission.to_csv('Submission.csv', index=False)

[0.15065022 0.37883891 0.47178786 ... 0.27819795 0.02921871 0.03736134]
       encounter_id  diabetes_mellitus
371          135000           0.312187
6208         135001           0.031436
3684         135002           0.144181
3687         135003           0.421604
10012        135004           0.018621
...             ...                ...
3724         145996           0.572121
9947         145997           0.305848
9751         145998           0.380083
2118         145999           0.255119
7894         146000           0.804198

[10234 rows x 2 columns]


In [146]:
"""
from sklearn.model_selection import GridSearchCV
param_1 ={'n_estimators': [10,20,50,80,90,100,120,130], 'learning_rate': [0.08, 0.1,0.12]}
param_2 ={'num_leaves':[60,64,70,80,90,100],'max_depth':[8,9,10,11]}
gbm = LGBMClassifier(random_state=999, tree_method = 'gpu_hist',boosting_type='gbdt', num_leaves=64)
grid_search = GridSearchCV(gbm, param_1, scoring='roc_auc',n_jobs=-1)
grid_search.fit(X_rdm,y_rdm)
grid_search.best_params_
"""

"\nfrom sklearn.model_selection import GridSearchCV\nparam_1 ={'n_estimators': [10,20,50,80,90,100,120,130], 'learning_rate': [0.08, 0.1,0.12]}\nparam_2 ={'num_leaves':[60,64,70,80,90,100],'max_depth':[8,9,10,11]}\ngbm = LGBMClassifier(random_state=999, tree_method = 'gpu_hist',boosting_type='gbdt', num_leaves=64)\ngrid_search = GridSearchCV(gbm, param_1, scoring='roc_auc',n_jobs=-1)\ngrid_search.fit(X_rdm,y_rdm)\ngrid_search.best_params_\n"

In [147]:
"""
features = X_train.columns
print(f"AUC is {roc_auc_score(y_valid, grid_search.predict(X_valid))}")
print(metrics.classification_report(y_valid, grid_search.predict(X_valid), labels=[0, 1]))
"""

'\nfeatures = X_train.columns\nprint(f"AUC is {roc_auc_score(y_valid, grid_search.predict(X_valid))}")\nprint(metrics.classification_report(y_valid, grid_search.predict(X_valid), labels=[0, 1]))\n'